In [1]:
import gurobipy as gp
from gurobipy import GRB

def solve_mixed_explanation(name_winner, name_loser, notes_winner, notes_loser, weights, subjects):
    """
    Résout le problème d'explication mixte : autorise les structures (1-m) OU (m-1).
    """
    # 1. Calcul des contributions
    deltas = []
    for k in range(len(subjects)):
        d = weights[k] * (notes_winner[k] - notes_loser[k])
        deltas.append(d)
        
    P_indices = [i for i, d in enumerate(deltas) if d > 0]
    C_indices = [j for j, d in enumerate(deltas) if d < 0]
    
    print(f"\n=== Comparaison {name_winner} > {name_loser} ===")
    print(f"Pros: {[(subjects[i], deltas[i]) for i in P_indices]}")
    print(f"Cons: {[(subjects[j], deltas[j]) for j in C_indices]}")
    
    if not C_indices:
        print("Aucun point négatif à expliquer.")
        return

    # 2. Modèle
    m = gp.Model("Mixed_Explanation")
    m.setParam('OutputFlag', 0)
    
    # --- Variables ---
    # Pour le type (1-m) : Pro i est le "Hub" qui couvre les Cons j (Leaves)
    # x_1m[i,j] = 1 si i couvre j dans un schéma (1-m)
    x_1m = {} 
    
    # Pour le type (m-1) : Con j est le "Hub" couvert par les Pros i (Leaves)
    # x_m1[i,j] = 1 si i aide à couvrir j dans un schéma (m-1)
    x_m1 = {}

    for i in P_indices:
        for j in C_indices:
            x_1m[i, j] = m.addVar(vtype=GRB.BINARY, name=f"1m_{subjects[i]}_{subjects[j]}")
            x_m1[i, j] = m.addVar(vtype=GRB.BINARY, name=f"m1_{subjects[i]}_{subjects[j]}")

    # Indicateurs de rôle "Hub"
    # h_P[i] = 1 si le Pro i est utilisé comme centre d'un (1-m)
    h_P = {i: m.addVar(vtype=GRB.BINARY) for i in P_indices}
    
    # h_C[j] = 1 si le Con j est utilisé comme centre d'un (m-1)
    h_C = {j: m.addVar(vtype=GRB.BINARY) for j in C_indices}

    # --- Contraintes ---

    # 1. Partition des Cons (Tout Con doit être couvert exactement une fois)
    # Un Con j est soit une "feuille" d'un (1-m), soit un "hub" d'un (m-1)
    for j in C_indices:
        # Somme des liens entrants (1-m) + Est-ce un hub (m-1) ?
        m.addConstr(gp.quicksum(x_1m[i, j] for i in P_indices) + h_C[j] == 1, name=f"Cover_{j}")

    # 2. Disjonction des Pros (Tout Pro utilisé au max une fois)
    # Un Pro i est soit un "hub" d'un (1-m), soit une "feuille" d'un (m-1), soit inutilisé
    for i in P_indices:
        # Est-ce un hub (1-m) + Somme des liens sortants (m-1)
        m.addConstr(h_P[i] + gp.quicksum(x_m1[i, j] for j in C_indices) <= 1, name=f"Use_{i}")

    # 3. Cohérence logique des liens
    # Si lien (1-m) i->j existe, alors i doit être déclaré Hub
    for i in P_indices:
        for j in C_indices:
            m.addConstr(x_1m[i, j] <= h_P[i])
            
    # Si lien (m-1) i->j existe, alors j doit être déclaré Hub
    for i in P_indices:
        for j in C_indices:
            m.addConstr(x_m1[i, j] <= h_C[j])

    # 4. Validité Financière (Somme pondérée > 0)
    
    # Pour chaque Pro Hub (Structure 1-m)
    # delta_i + sum(delta_j * x_1m_ij) >= 0
    for i in P_indices:
        contrib_cons = gp.quicksum(deltas[j] * x_1m[i, j] for j in C_indices)
        # On multiplie par h_P[i] pour désactiver la contrainte si pas hub (big-M ou juste logique 0>=0)
        # Ici : delta_i * h_P[i] + contrib_cons >= 0 (car x_ij est 0 si h_P est 0)
        m.addConstr(deltas[i] * h_P[i] + contrib_cons >= 0, name=f"Valid_1m_{i}")

    # Pour chaque Con Hub (Structure m-1)
    # sum(delta_i * x_m1_ij) + delta_j >= 0
    for j in C_indices:
        contrib_pros = gp.quicksum(deltas[i] * x_m1[i, j] for i in P_indices)
        m.addConstr(contrib_pros + deltas[j] * h_C[j] >= 0, name=f"Valid_m1_{j}")

    # Résolution
    m.optimize()

    # --- Affichage ---
    if m.Status == GRB.OPTIMAL:
        print(">>> SUCCÈS : Explication mixte trouvée !")
        print("Détails :")
        
        # Affichage des structures (1-m)
        for i in P_indices:
            if h_P[i].X > 0.5:
                cons_covered = [subjects[j] for j in C_indices if x_1m[i, j].X > 0.5]
                val_cons = sum(deltas[j] for j in C_indices if x_1m[i, j].X > 0.5)
                print(f"  [Type 1-m] L'avantage {subjects[i]} (+{deltas[i]}) compense {cons_covered} ({val_cons})")

        # Affichage des structures (m-1)
        for j in C_indices:
            if h_C[j].X > 0.5:
                pros_used = [subjects[i] for i in P_indices if x_m1[i, j].X > 0.5]
                val_pros = sum(deltas[i] for i in P_indices if x_m1[i, j].X > 0.5)
                print(f"  [Type m-1] Les avantages {pros_used} (+{val_pros}) compensent {subjects[j]} ({deltas[j]})")
                
    elif m.Status == GRB.INFEASIBLE:
        print(">>> ÉCHEC : Impossible de trouver une explication mixte valide.")
    else:
        print(f"Statut : {m.Status}")

# --- Données ---
matieres = ["Anatomie", "Biologie", "Chirurgie", "Diagnostic", "Epidemiologie", "Forensic", "Génétique"]
weights = [8, 7, 7, 6, 6, 5, 6]

# Cas 1 : z > t (On s'attend à un Succès mixte)
z_notes = [74, 89, 74, 81, 68, 84, 79]
t_notes = [74, 71, 84, 91, 77, 76, 73]
solve_mixed_explanation("z", "t", z_notes, t_notes, weights, matieres)

# Cas 2 : a1 > a2 (Nouveaux candidats)
# a1: 89, 74, 81, 68, 84, 79, 77
# a2: 71, 84, 91, 79, 78, 73.5, 77
a1_notes = [89, 74, 81, 68, 84, 79, 77]
a2_notes = [71, 84, 91, 79, 78, 73.5, 77] # Notez le 73.5 pour Forensic
solve_mixed_explanation("a1", "a2", a1_notes, a2_notes, weights, matieres)


=== Comparaison z > t ===
Pros: [('Biologie', 126), ('Forensic', 40), ('Génétique', 36)]
Cons: [('Chirurgie', -70), ('Diagnostic', -60), ('Epidemiologie', -54)]
Set parameter Username
Set parameter LicenseID to value 2755075
Academic license - for non-commercial use only - expires 2026-12-15
>>> SUCCÈS : Explication mixte trouvée !
Détails :
  [Type 1-m] L'avantage Biologie (+126) compense ['Diagnostic', 'Epidemiologie'] (-114)
  [Type m-1] Les avantages ['Forensic', 'Génétique'] (+76) compensent Chirurgie (-70)

=== Comparaison a1 > a2 ===
Pros: [('Anatomie', 144), ('Epidemiologie', 36), ('Forensic', 27.5)]
Cons: [('Biologie', -70), ('Chirurgie', -70), ('Diagnostic', -66)]
>>> ÉCHEC : Impossible de trouver une explication mixte valide.
